In [2]:


# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Preprocesado y modelado
# ==============================================================================
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.weightstats import ttest_ind

# Configuración matplotlib
# ==============================================================================
plt.rcParams['image.cmap'] = "bwr"
#plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')




In [71]:
data_frame = pd.read_csv('data.csv')

In [72]:
data_frame.head(475)

,SEMESTRE ACTUAL,ESTRATO,PROMEDIO CARRERA,PROMEDIO SEMESTRE,ESTADO
0,10,1,2.833,3.50,ACTIVO
1,10,3,2.542,0.00,ACTIVO
2,6,1,0.000,0.00,INACTIVO
3,3,2,3.375,3.70,INACTIVO
4,2,3,2.541,3.05,ACTIVO
...,...,...,...,...,...
470,1,1,0.000,0.00,ACTIVO
471,1,1,0.000,0.00,ACTIVO
472,1,3,0.000,0.00,ACTIVO
473,1,2,0.000,0.00,ACTIVO


## Variables de interes 
 semestre actual, Estrato, Promedio de carrera, Promedio de semestre, Estado, Resultado 


# Desarollo 
En esta parte nos interesa explorar los datos y desarrollar el modelo sde regresion logistica, en el estudio de la Desercion estudiantil en el programa de matematicas.

# Exploracion de datos 
vamos a crear los arreglos con los datos de cada una d elas variables de interes 

In [81]:


#array de ESTRATO
#estrato = data_frame['ESTRATO'].values

#array de PROMEDIO CARRERA
promCarr = data_frame['PROMEDIO CARRERA'].values

#array de PROMEDIO SEMESTRE
promSeme = data_frame['PROMEDIO SEMESTRE'].values

#array de RESUTADO
#resultados = data_frame['RESULTADO'].values



Se codifica la variable respuesta como 1 si esta inactivo y 0 silo esta, y se identifica cuantas observaciones hay de cada clase.

In [74]:

data_frame['ESTADO'] = np.where(data_frame['ESTADO'] == 'INACTIVO', 1, 0)

print("Número de observaciones por clase")
print(data_frame['ESTADO'].value_counts())
print("")

print("Porcentaje de observaciones por clase")
print(100 * data_frame['ESTADO'].value_counts(normalize=True))



Número de observaciones por clase
0    253
1    222
Name: ESTADO, dtype: int64

Porcentaje de observaciones por clase
0    53.263158
1    46.736842
Name: ESTADO, dtype: float64


El 53,26% de los estudiantes esta activo y el 46.73% estan inactivos, y estos son los datos en lo que tenemos interes. Un modelo de clasificación que sea útil debe de ser capaz de predecir correctamente un porcentaje de observaciones por encima del porcentaje de la clase mayoritaria. En este caso, el umbral de referencia que se tiene que superar es del 53,26%

# Ajuste del modelo

Se ajusta un modelo de regresión logística múltiple con el objetivo de predecir si un estudiante esta  inactivo en función de todas las variables disponibles.

In [75]:
# Creación de la matriz de predictores
X = data_frame.drop(columns = 'ESTADO')

# Creación de la variable dependiente
y = data_frame['ESTADO']

# A la matriz de predictores se le tiene que añadir una columna de 1s para el intercept del modelo
X = sm.add_constant(X, prepend = True)

# Creación del modelo de regresión logística
modelo = sm.Logit(endog = y, exog = X)

# Entrenamiento del modelo
modelo = modelo.fit()

# Resumen del modelo
print(modelo.summary())




Optimization terminated successfully.
         Current function value: 0.525773
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 ESTADO   No. Observations:                  475
Model:                          Logit   Df Residuals:                      470
Method:                           MLE   Df Model:                            4
Date:                Tue, 05 Dec 2023   Pseudo R-squ.:                  0.2391
Time:                        15:51:19   Log-Likelihood:                -249.74
converged:                       True   LL-Null:                       -328.23
Covariance Type:            nonrobust   LLR p-value:                 6.492e-33
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.0004      0.318      3.144      0.002       0.377       1.624
SEMESTRE

# Predicciones


Una vez entrenado el modelo, se pueden obtener predicciones para nuevos datos. Los modelos de statsmodels permiten calcular los intervalos de confianza asociados a cada predicción.

In [76]:
 #Predicciones con intervalo de confianza 
# ==============================================================================
predicciones = modelo.predict(exog = X)

# Clasificación predicha
# ==============================================================================
clasificacion = np.where(predicciones<0.5, 0, 1)
clasificacion

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,

# Accuracy de test


Se calcula el porcentaje de aciertos que tiene el modelo al predecir las observaciones de test (accuracy).

In [78]:
# Accuracy del modelo 
# ==============================================================================
# Comparación con las etiquetas reales
accuracy = np.mean(clasificacion == y)

# Mostrar la precisión
print(f'Precisión del modelo: {accuracy * 100:.2f}%')

Precisión del modelo: 74.11%


In [80]:


# Matriz de confusión de las predicciones de test
# ==============================================================================
confusion_matrix = pd.crosstab(
    y.ravel(),
    clasificacion,
    rownames=['Real'],
    colnames=['Predicción']
)
confusion_matrix



Predicción,0,1
Real,,
0,197,56
1,67,155


# verificacion de datos 

In [90]:
# importamos la bases de datos ocn los resultados

datares = pd.read_csv('dataconresultado.csv')
datares.head(475)

#array de RESUTADO para comparar
resultados = datares['RESULTADO'].values

x1 = data_frame['SEMESTRE ACTUAL'].values

#array de ESTRATO
x2 = data_frame['ESTRATO'].values

#array de PROMEDIO CARRERA
x3 = data_frame['PROMEDIO CARRERA'].values

#array de PROMEDIO SEMESTRE
x4 = data_frame['PROMEDIO SEMESTRE'].values

#array de RESUTADO
#resultados = data_frame['RESULTADO'].values

b0 = 1.0004
b1 = -0.2967
b2 = 0.3380
b3 = -0.5611
b4 = 0.2385

# Inicializa un array vacío para almacenar los resultados
predicciones = []

# Calcula el sigmoide para cada conjunto de valores
for i in range(len(x1)):
    sigmoide_i = 1 / (1 + np.exp(-(b0 + b1 * x1[i] + b2 * x2[i] + b3 * x3[i] + b4*x4)))
    predicciones.append(sigmoide_i)

# Convierte la lista de resultados en un array de NumPy
PREDICCIONES = np.array(predicciones)

RESULTADOS_pred = np.where(PREDICCIONES < 0.5, 0, 1)

print("Resultado predicho \n\n",RESULTADOS_pred,'\n')
print('Resultados reales \n\n',resultados)        


Resultado predicho 

 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]] 

Resultados reales 

 [0 0 1 1 0 1 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0
 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1
 1 1 1 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 0 0 0
 0 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 0 0 1 0 0
 1 1 1 1 0 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 0 1 0
 1 1 0 1 0 1 1 1 1 0 0 1 0 1 0 1 1 0 0 0 1 0 1 1 0 0 1 0 1 0 0 1 1 1 1 1 0
 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 0 0 0 1 1 0 0 1 1 1 1 1 1 1 1
 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0
 0 1 0 1 1 0 0 0 0 0 0 1 1 0 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1 0 1 1 1 0 0 0 1
 0 1 0 1 0 0 1 1 0 0 1 1 1 1 0 0 0 0 1 0 0 0 1 1 1 0 1 1 1 0 1 1 0 0 1 1 0
 0 0 0 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 0 1 1 1 0 1 0 1 0 0 1 1
 0 1 0 1 0

In [94]:
from sklearn.metrics import f1_score

# Calcula el F1 Score
f1 = f1_score(resultados, clasificacion)

print(f"F1 Score: {f1}")

F1 Score: 0.7159353348729791
